In [ ]:
# Import necessary libraries
from pathlib import Path
import json, random
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
from PIL import Image

# Ensures predictable results
seed = 42
random.seed(42)
torch.manual_seed(seed)

# Configure Torch device
device = "cpu"
#device = "cuda" if torch.cuda_is_available() else "cpu"
print(f"Using {device}")

In [ ]:
# Define DogsDataset class
class DogsDataset(Dataset):
    def __init__(self, jsonl_path, transform=None, class_to_idx=None):
        # Define path to data
        self.path = Path(jsonl_path)
        # Quite literally transforming the image
        self.transform = transform

        # Load JSONL
        with self.path.open() as f:
            # Confusing JSONL loading
            self.items = [json.loads(line) for line in f if line.strip()]
        
        # Create or assign Class Mapping (e.g. "0" = Chihuahua)
        if class_to_idx is None:
            # Ensures deterministic class order
            classes = sorted({x["class_name"] for x in self.items})
            # Assigns a number to each class
            self.class_to_idx = {c: i for i, c in enumerate(classes)}
        else:
            self.class_to_idx = class_to_idx
    
    # Returns No. of data points
    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item = self.items[idx]

        # Convert to RGB image
        img = Image.open(item["image"]).convert("RGB")
        label = self.class_to_idx[item["class_name"]]

        if self.transform:
            img = self.transform(img)
        
        return img, label

In [ ]:
# Image Augmentation
image_size = 224
stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

# Make training data harder for better learning
train_tf = T.Compose([
    T.RandomResizedCrop(image_size),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(*stats)
])

# Validation/Test data gets standardized
eval_tf = T.Compose([
    T.Resize(int(image_size * 1.14)),
    T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(*stats)
])

In [ ]:
# Define Datasets and DataLoaders 
train_ds = DogsDataset("data/splits/dogs_train.jsonl", transform=train_tf)
# To ensure consistency the same class_to_idx has been used in all datasets
val_ds = DogsDataset("data/splits/dogs_val.jsonl", transform=eval_tf, class_to_idx=train_ds.class_to_idx)
test_ds = DogsDataset("data/splits/dogs_test.jsonl", transform=eval_tf, class_to_idx=train_ds.class_to_idx)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

In [ ]:
# Sets initial weights to those from the ResNet18 model
weights = torchvision.models.ResNet18_Weights.DEFAULT
model = torchvision.models.resnet18(weights=weights)

# Replace the Fully Connected layer with the dogs classes
num_classes = len(train_ds.class_to_idx)
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device) # Move model to device CPU/GPU

In [ ]:
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4) # 3e-4 is a "Kaparthy Constant" (very safe learning rate)

# Learning Rate Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.1) # Decreases learning rate when accuracy "plateaus"

In [ ]:
# Training Loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # Sets model to training mode, enabling Dropout/Batchnorm

    current = 0
    loss_list = []
    for batch, (X, y) in enumerate(dataloader):
        # X is the input, y is the answer
        X, y = X.to(device), y.to(device) # Move to device

        pred = model(X) # Forward pass essentially just run the inputs through the nn
        loss = loss_fn(pred, y) # Figure out loss based on the prediction (pred) and the answer (y)
        loss.backward() # Backpropogation: go back and figure out which weights effect the loss the most
        optimizer.step() # Tweak the weights down the gradient or something COME BACK TO THIS
        optimizer.zero_grad() # Resets gradients

        current += len(X)
        loss_value = loss.item()
        loss_list.append(loss_value)
        print(f"Loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]", end='\r', flush=True)
    print(f"Average Loss: {sum(loss_list) / len(loss_list):>7f}  [{size:>5d}/{size:>5d}]", end='\r', flush=True)
    print()

# Testing Loop
def test_loop(dataloader, model, loss_fn):
    model.eval() # Sets model to evaluation mode, disabling Dropout/Batchnorm
    test_loss, correct = 0, 0

    with torch.no_grad(): # No gradients are required, saves memory
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) # Move to device
            pred = model(X)

            # Sums up the correct predictions, pred.argmax(1) gets the index of the highest probability
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    print(f"Accuracy: {correct / len(dataloader.dataset):.01%}")
    return correct / len(dataloader.dataset)

# Train
acc = 0
epochs = 25
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-----------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    acc = test_loop(val_loader, model, loss_fn)
    print()
    scheduler.step(acc)
print("Done Training!")

In [ ]:
torch.save({
    "state_dict": model.state_dict(),
    "class_to_idx": train_ds.class_to_idx
}, f"resnet18_dogs_cpu_{(acc*100):.1f}.pth")